In [4]:
##
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import math
import os

In [ ]:
##统一文件的表头
#!/user/bin/env python3
# -*- coding: utf-8 -*-
import os
import re
import datetime
import numpy as np
import pandas as pd

starttime = datetime.datetime.now()	# 运行时间统计
if __name__ == '__main__':
    filenames_in = r'D:\项目\扬州\公交刷卡数据\20210501-0516旧卡（交易时间需减8小时看）'  # 输入文件的文件地址
    filenames_out = r'D:\项目\扬州\公交刷卡数据\旧卡'  # 新文件的地址
    pathDir = os.listdir(filenames_in)
    for allDir in pathDir:
        child = re.findall(r"(.+?).xlsx", allDir)  # 正则的方式读取文件名，去扩展名
        if len(child) >= 0:  # 去掉没用的系统文件
            newfile = ''
            needdate = child  #### 这个就是所要的文件名
        domain1 = os.path.abspath(filenames_in)  # 待处理文件位置
        info = os.path.join(domain1, allDir)  # 拼接出待处理文件名字

        # ------------数据处理过程---------------
        print(info, "开始处理")
        # df = pd.DataFrame(pd.read_csv(info, index_col=0))   # 读取文件数据，以第一列为索引
        # df = df.iloc[:, :-1]    # 删除最后一列数据
        sub_GPS = pd.read_excel(info,header=None,names=['UP_TIME', 'CORP_CODE', '卡内号', '卡号', 'CC_CODE', '车载机号', 'BUSINESS_ID',
       '车辆号', 'TERM_TRAN_NO', 'OFF_TRAN_NO', '交易日期', '交易时间', 'TRAN_TYPE',
       'TRAN_CCY', '交易金额', 'TRAN_TAC', 'TRAN_STATUS', 'REMARKS', '卡类型', '城市代码',
       'TRAN_BAL', 'TRAN_COUNT', 'FAIL_WHYS', 'DEAL_TYPE', '线路', 'CARD_SEQNO',
       'CONSUMER_CITY', 'FILE_NAME', 'ROW_NUM', 'LAST_ACCT_BAL',
       'LAST_POUS_COUNT', 'LAST_POUS_TIME', 'LAST_POUS_AMT', 'BALANCE_TIME',
       'SD_CARD_NO', 'SD_CARD_FACE_NO', 'COND_CARD_NO', 'COND_CARD_FACE_NO',
       'SD_TRAN_TIME', 'SP_TRAN_TIME', 'SD_AREA_CODE', 'SD_BUSINESS_ID',
       'SD_CC_CODE', 'SD_STATION_ID', 'SD_LINE_ID', 'SD_BUS_NO',
       'SD_TERM_TYPE', 'IS_DISPOSED', 'IS_TRANS', 'DRIVER_IC_CARD_NO',
       'DRIVER_UP_TIME', 'LINE_PRICE', 'BUS_FILE_HEAD_TIME',
       'BUS_FILE_CARD_NO_SJ', 'BUS_FILE_CARD_NO_SPY', 'BUS_FILE_TIME',
       'CARD_TYPE', 'SAMID', 'SAL_CHECK_NO', 'CHECK_TAC', 'SP_POSE_MANUNO',
       'SP_OPTION_NO', 'SP_TRAN_COUNT'])
        GPS_22=sub_GPS[['UP_TIME', 'CORP_CODE', '卡内号', '卡号', 'CC_CODE', '车载机号', 'BUSINESS_ID',
       '车辆号', 'TERM_TRAN_NO',  '交易日期', '交易时间',  '卡类型', '城市代码', '线路']]###进行所有子文件夹内目标数据的提取
        df = pd.DataFrame(GPS_22)
        domain2 = os.path.abspath(filenames_out)  # 处理完文件保存地址
        outfo = os.path.join(domain2, allDir)  # 拼接出新文件名字
        df.to_excel(outfo, index=False)
        print(info, "处理完")

endtime = datetime.datetime.now()
print(endtime - starttime)

In [ ]:
import glob
##母文件夹地址和文件类型
files=glob.glob(r'D:\项目\扬州\公交刷卡数据\旧卡/*.xlsx')
print(files)
##需要读取的列
cols=['UP_TIME', 'CORP_CODE', '卡内号', '卡号', 'CC_CODE', '车载机号', 'BUSINESS_ID',
       '车辆号', 'TERM_TRAN_NO',  '交易日期', '交易时间',  '卡类型', '城市代码', '线路']
##列表推导出对象
dflist=[pd.read_excel(i,usecols=cols) for i in files]
print(dflist)
##合并
df=pd.concat(dflist)

In [ ]:
df=df.loc[df.UP_TIME!='UP_TIME']##清除表头
df

In [ ]:
laoka=df[['车载机号', '车辆号', '交易日期', '交易时间',  '卡类型',  '线路']]
laoka

In [ ]:
#切分字符串
laoka['交易时间']= laoka['交易时间'].astype(str)
d=laoka['交易时间'].str.split('',expand=True)#切割符号
d['时1']=d.loc[:,[9]]
d['时2']=d.loc[:,[10]]
d['分钟1']=d.loc[:,[11]]##字符串挨个索引
d['分钟2']=d.loc[:,[12]]
d['秒1']=d.loc[:,[13]]
d['秒2']=d.loc[:,[14]]
laoka['时']=d['时1']+d['时2']
laoka['分钟']=d['分钟1']+d['分钟2']###字符串直接拼接
laoka['秒']=d['秒1']+d['秒2']
laoka
laoka[['交易日期','时','分钟','秒']]= laoka[['交易日期','时','分钟','秒']].apply(pd.to_numeric)
##8小时修正
kaoka1=laoka[laoka['时']>=8]
kaoka1['时']=kaoka1['时']-8
kaoka2=laoka[laoka['时']<8]
kaoka2['交易日期']=kaoka2['交易日期']-1
kaoka2['时']=kaoka2['时']+24-8
xiuzhenglaoka=kaoka1.append(kaoka2)
xiuzhenglaoka
##重新合并时分秒
xiuzhenglaoka[['时','分钟','秒']]= xiuzhenglaoka[['时','分钟','秒']].astype(str)
xiuzhenglaoka['时间']=xiuzhenglaoka['时']+':'+xiuzhenglaoka['分钟']+':'+xiuzhenglaoka['秒']
xiuzhenglaoka

In [ ]:
##分别切割出车辆编号
xiuzhenglaoka['车辆号']= xiuzhenglaoka['车辆号'].astype(str)
d=xiuzhenglaoka['车辆号'].str.split('',expand=True)#切割符号
d['CHEID1']=d.loc[:,[3]]##字符串挨个索引
d['CHEID2']=d.loc[:,[4]]##字符串挨个索引
d['CHEID3']=d.loc[:,[5]]##字符串挨个索引
d['CHEID4']=d.loc[:,[6]]##字符串挨个索引
xiuzhenglaoka['CHEID']=d['CHEID1']+d['CHEID2']+d['CHEID3']+d['CHEID4']###字符串直接拼接
xiuzhenglaoka['车载机号']= xiuzhenglaoka['车载机号'].astype(str)
d=xiuzhenglaoka['车载机号'].str.split('',expand=True)#切割符号
d['CHEID1']=d.loc[:,[5]]##字符串挨个索引
d['CHEID2']=d.loc[:,[6]]##字符串挨个索引
d['CHEID3']=d.loc[:,[7]]##字符串挨个索引
d['CHEID4']=d.loc[:,[8]]##字符串挨个索引
xiuzhenglaoka['车载机号ID']=d['CHEID1']+d['CHEID2']+d['CHEID3']+d['CHEID4']###字符串直接拼接
xiuzhenglaoka

In [ ]:
laokac=xiuzhenglaoka[['CHEID','车载机号ID','交易日期','时间']]
laokac

In [8]:
laokacc=laokac.loc[laokac.交易日期==20210513]
laokacc

,Unnamed: 0,CHEID,车载机号ID,交易日期,时间
912095,1,4879.0,2853,20210513,10:45:30
912096,2,4879.0,2853,20210513,10:51:13
912097,3,4879.0,2853,20210513,11:1:29
912098,4,4879.0,2853,20210513,11:4:8
912099,5,4879.0,2853,20210513,11:10:17
...,...,...,...,...,...
1544228,92179,5143.0,4986,20210513,17:27:59
1544477,93205,6183.0,5378,20210513,16:37:11
1544478,93206,6183.0,5378,20210513,16:37:12
1544479,93207,6183.0,5378,20210513,16:44:58


In [ ]:
laokac.to_csv(r'D:\项目\扬州\公交刷卡数据\老卡时间修正0606-0619.csv')

In [7]:
laokac=pd.read_csv('D:\项目\扬州\公交刷卡数据\老卡时间修正0606-0619.csv')

In [9]:
laokacc.to_csv(r'D:\项目\扬州\公交刷卡数据\老卡时间修正5月13号.csv')

In [ ]:
xinkashijian=pd.read_excel(r'D:\项目\扬州\公交刷卡数据\20210501-0516新卡new/20210501.xlsx')
xinkashijian

In [ ]:
xinkashijian.columns

In [ ]:
xinkashijian[['TXN_AMT', 'CRD_BAL_BEF', 'CRD_BAL_AFT', 'SET_DT', 'FEE_TP']]

In [ ]:
import glob
##母文件夹地址和文件类型
files=glob.glob(r'D:\项目\扬州\公交刷卡数据\20210501-0516新卡new/*.xlsx')
print(files)
##需要读取的列
cols=['EXTEND6','TXN_DT']
##列表推导出对象
dflist=[pd.read_excel(i,usecols=cols) for i in files]
print(dflist)
##合并
df2=pd.concat(dflist)
df2

In [ ]:
df2=df2.loc[df2.TXN_DT!='TXN_DT']##清除表头
df2

In [ ]:
#时间切割
df2['TXN_DT']= df2['TXN_DT'].astype(str)
d=df2['TXN_DT'].str.split('',expand=True)#切割符号
d['日期1']=d.loc[:,[1]]
d['日期2']=d.loc[:,[2]]
d['日期3']=d.loc[:,[3]]
d['日期4']=d.loc[:,[4]]
d['日期5']=d.loc[:,[5]]
d['日期6']=d.loc[:,[6]]
d['日期7']=d.loc[:,[7]]
d['日期8']=d.loc[:,[8]]
d['时1']=d.loc[:,[9]]
d['时2']=d.loc[:,[10]]
d['分钟1']=d.loc[:,[11]]##字符串挨个索引
d['分钟2']=d.loc[:,[12]]
d['秒1']=d.loc[:,[13]]
d['秒2']=d.loc[:,[14]]
df2['日期']=d['日期1']+d['日期2']+d['日期3']+d['日期4']+d['日期5']+d['日期6']+d['日期7']+d['日期8']
df2['时']=d['时1']+d['时2']
df2['分钟']=d['分钟1']+d['分钟2']###字符串直接拼接
df2['秒']=d['秒1']+d['秒2']
##重新合并时分秒
df2[['时','分钟','秒']]= df2[['时','分钟','秒']].astype(str)
df2['时间']=df2['时']+':'+df2['分钟']+':'+df2['秒']
#切分字符串
df2['EXTEND6']= df2['EXTEND6'].astype(str)
d=df2['EXTEND6'].str.split('',expand=True)#切割符号
d['CHEID1']=d.loc[:,[3]]##字符串挨个索引
d['CHEID2']=d.loc[:,[4]]##字符串挨个索引
d['CHEID3']=d.loc[:,[5]]##字符串挨个索引
d['CHEID4']=d.loc[:,[6]]##字符串挨个索引
df2['CHEID']=d['CHEID1']+d['CHEID2']+d['CHEID3']+d['CHEID4']###字符串直接拼接
df2

In [ ]:
xinkac=df2[['CHEID','日期','时间']]
xinkac

In [12]:
xinkacc=xinkac.loc[xinkac.日期==20210513]
xinkacc

,Unnamed: 0,CHEID,日期,时间
1092808,0,6260.0,20210513,06:48:25
1092809,1,6260.0,20210513,06:36:13
1092810,2,6260.0,20210513,05:56:17
1092811,3,1336.0,20210513,10:04:04
1092812,4,6067.0,20210513,10:12:20
...,...,...,...,...
1189647,96839,4152.0,20210513,17:48:19
1189648,96840,4152.0,20210513,18:19:48
1189649,96841,4152.0,20210513,17:13:19
1189650,96842,4152.0,20210513,17:13:18


In [10]:
xinkac=pd.read_csv(r'D:\项目\扬州\公交刷卡数据\新卡不必时间修正0606-0619.csv')

In [ ]:
xinkac.to_csv(r'D:\项目\扬州\公交刷卡数据\新卡不必时间修正0606-0619.csv')

In [13]:
xinkacc.to_csv(r'D:\项目\扬州\公交刷卡数据\新卡不必时间修正5月13日.csv')

In [14]:
##导入刷卡数据
laokac=pd.read_csv(r'D:\项目\扬州\公交刷卡数据\老卡时间修正5月13号.csv')
laokac=laokac[['CHEID','交易日期','时间']]
laokac.columns=[['CHEID','日期','时间']]
laokac.to_csv(r'D:\项目\扬州\公交刷卡数据\老卡时间修正5月13号.csv')
xinkac=pd.read_csv(r'D:\项目\扬州\公交刷卡数据\新卡不必时间修正5月13日.csv')
xinkac=xinkac[['CHEID','日期','时间']]
xinkac.to_csv(r'D:\项目\扬州\公交刷卡数据\新卡不必时间修正5月13日.csv')